In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd
df_test = pd.read_csv(
    "/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2021_train.tsv", 
    sep='\t', 
    header=0,
    nrows=1000)
df_quiz = pd.read_csv(
    "/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2021_quiz.tsv",  
    sep='\t', 
    header=0,
    nrows=1000)

#Standardize all payment times to EST (or common time zone)

# Payment time - Delivery time = delivary dates 

In [36]:
df_test[0:5]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


In [76]:
from datetime import datetime

df1 = pd.DataFrame(df_test,columns=[
  "declared_handling_days","carrier_min_estimate","payment_datetime", "delivery_date"
  ])

df_test["payment_datetime"] = pd.to_datetime(df_test["payment_datetime"], utc=True).dt.date
df_test["delivery_date"] = pd.to_datetime(df_test["delivery_date"]).dt.date

df_test_clean = pd.DataFrame()
df_test_clean["estimate"] = df_test["declared_handling_days"] + df_test["carrier_min_estimate"]
df_test_clean["result"] = df_test["delivery_date"] - df_test["payment_datetime"]
df_test_clean["result"] = (df_test_clean["result"]).dt.days

df_test_clean[0:5]

,estimate,result
0,6.0,5
1,5.0,4
2,4.0,4
3,4.0,5
4,4.0,3


In [12]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from nltk import sent_tokenize
from tqdm import tqdm
from scipy import sparse
import pickle
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import (
    LogisticRegression,
    SGDClassifier,
)
from sklearn.neural_network import MLPClassifier

In [16]:
classifiers = [("BernoulliNB", BernoulliNB()),
        ("ComplementNB", ComplementNB()),
        ("MultinomialNB", MultinomialNB(        )),
        ("KNeighborsClassifier", KNeighborsClassifier()),
        ("DecisionTreeClassifier", DecisionTreeClassifier()),
        ("RandomForestClassifier", RandomForestClassifier()),
        ("LogisticRegression", LogisticRegression(max_iter=1000)),
        ("SGDClassifier", SGDClassifier()),
        ("AdaBoostClassifier", AdaBoostClassifier()),
        ("MLPClassifier", MLPClassifier(max_iter=1000)),
]

